In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install -r requirements.txt

## Build dataset

In [ ]:
from src.surfnet_train.data.data_processing import get_annotations_from_files, get_annotations_from_db
from src.surfnet_train.data.data_processing import generate_yolo_files, get_train_valid, build_yolo_annotations_for_images
from pathlib import Path

In [ ]:
from argparse import Namespace
args = Namespace(
    data_dir="/home/charles/Programs/surfrider/yolov5_train_test/data/test_data/",
    images_dir="/home/charles/Programs/surfrider/surfnet/data/images/images/",
    bbox_filename="bounding_boxes_202206171635.csv",
    images_filename="images_for_labelling_202206171635.csv",
    limit_data=40
)

In [ ]:
df_bboxes, df_images = get_annotations_from_files(Path(args.data_dir),
                                                  args.bbox_filename,
                                                  args.images_filename)

In [ ]:
yolo_filelist, cpos, cneg = build_yolo_annotations_for_images(args.data_dir, args.images_dir, 
                                                              df_bboxes, df_images, args.limit_data)

In [ ]:
# split train test
train_files, val_files = get_train_valid(yolo_filelist)

In [ ]:
# generates data.yaml, train.txt, val.txt
generate_yolo_files(args.data_dir, train_files, val_files)

## Training

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%%writefile yolov5/data/hyps/hyp.scratch.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.012  # image HSV-Hue augmentation (fraction)
hsv_s: 0.6  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.3  # image HSV-Value augmentation (fraction)
degrees: 2.0  # image rotation (+/- deg)
translate: 0.3  # image translation (+/- fraction)
scale: 0.25  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.1  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.7  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

A model trained with higher resolution images can have better results for detecting small objects

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 32 --epochs 30 --data "data.yaml" --weights "yolov5s.pt" --workers 23 --project "ben" --name "yolo_ben_30_I&L" --exist-ok 

In [ ]:
ls

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 20 --epochs 30 --data "data.yaml" --weights "yolov5s.pt" --workers 23 --project "ben" --name "yolo_ben_1" --exist-ok 

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 64 --epochs 100 --data "data.yaml" --weights "yolov5s.pt" --workers 23 --project "ben" --name "yolo_ben_100" --exist-ok --device 0

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 32 --epochs 50 --data "data.yaml" --weights "yolov5s.pt" --workers 23 --project "ben" --name "yolo_ben_2" --exist-ok  --device 0

add smthg to device ; une des variables pas dans le GPU 

In [ ]:
!python "yolov5/train.py" --img 640 --hyp "yolov5/data/hyps/hyp.scratch.yaml" --batch 32 --epochs 10 0 --data "data.yaml" --weights "yolov5s.pt" --workers 23 -